In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
import copy
import random
import itertools

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several version of the prepare_users_df method and test which works best in your recommender.

In [3]:
def prepare_users_df(interactions_df):
    
    # Write your code here
    #  ---start---
    
    #     
    # users_df    
    #     
    # e.g. -> pd.DataFrame([[1], [4], [6]], columns=['user_id'])
    
    
    
#         'user_room_segment': interactions_df['room_segment'],
#         'user_n_people_bucket': interactions_df['n_people_bucket'],
#         'user_weekend_stay': interactions_df['weekend_stay']
#     'user_rate_plan': interactions_df['rate_plan']
#     'user_length_of_stay_bucket': interactions_df['length_of_stay_bucket']
#  'user_term': interactions_df['term']
    
    items_with_id = {
        'user_id': interactions_df['user_id'],
        'user_room_segment': interactions_df['room_segment']
    }  

    # print(users_df)
    user_features = pd.DataFrame(items_with_id)
    
    users_df = pd.get_dummies(user_features)
    # print(items_df)
#     users_df['user_id'] = interactions_df['user_id']
    #     
    # user_features    
    #
    
    # user_id = pd.DataFrame(interactions_df['user_id'], columns=['user_id'])
    
    # user_features = interactions_df.tolist()
    items_without_id = {
        'user_room_segment': interactions_df['room_segment']
    }  
        
#     user_features = users_df.columns.tolist()
    user_features = pd.get_dummies(pd.DataFrame(items_without_id)).columns.tolist()
    #     
    # print   
    #
    
    # print(users_df)
    # print(user_features)

    #  ---end---
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)


display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

print(user_features)

,user_id,user_room_segment_[0-160],user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_room_segment_[500-900]
0,1,0,0,1,0,0
52,50,0,1,0,0,0
57,50,0,0,1,0,0
94,1,0,1,0,0,0
95,1,0,1,0,0,0
103,96,1,0,0,0,0
150,115,0,1,0,0,0
151,115,0,1,0,0,0
187,115,0,1,0,0,0
188,115,0,1,0,0,0


['user_room_segment_[0-160]', 'user_room_segment_[160-260]', 'user_room_segment_[260-360]', 'user_room_segment_[360-500]', 'user_room_segment_[500-900]']


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into on-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because of the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features.

In [4]:
def prepare_items_df(interactions_df):
    
    # Write your code here
    #  ---start---
    
    #     
    # items_df -> a DataFrame containing one record per item_id with item_id column    
    # 
    
    #left_ids = rng.choice(interactions_df['item_id'], size=100, replace=False)
    #items_df = interactions_df.loc[interactions_df['item_id'].isin(left_ids)]
    
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html    
    # term, length_of_stay_bucket, rate_plan, room_segment, n_people_bucket, weekend_stay
        
        
#         'room_segment': interactions_df['room_segment'],
#         'n_people_bucket': interactions_df['n_people_bucket'],
#         'weekend_stay': interactions_df['weekend_stay']
#          'rate_plan': interactions_df['rate_plan']
# 'length_of_stay_bucket': interactions_df['length_of_stay_bucket']
#     'term': interactions_df['term']


    items_with_id = {
        'item_id': interactions_df['item_id'],
        'room_segment': interactions_df['room_segment']
    }    
        
    item_features = pd.DataFrame(items_with_id)
    
    items_df = pd.get_dummies(item_features)
    # print(items_df)
#     items_df['item_id'] = interactions_df['item_id']
    # print(items_df)
    
    #     
    # item_features -> numerical item feature columns   
    #    
# The column label 'item_id' is not unique.  

    items_without_id = {
        'room_segment': interactions_df['room_segment']
    }  
    
    item_features = pd.get_dummies(pd.DataFrame(items_without_id)).columns.tolist()
    
    #  ---end---
    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

print(item_features)

,item_id,room_segment_[0-160],room_segment_[160-260],room_segment_[260-360],room_segment_[360-500],room_segment_[500-900]
0,0,0,0,1,0,0
1,1,0,1,0,0,0
2,2,0,1,0,0,0
3,3,0,1,0,0,0
4,4,1,0,0,0,0
5,5,0,0,1,0,0
6,6,0,0,1,0,0
54,6,0,0,1,0,0
57,6,0,0,1,0,0
61,6,0,0,1,0,0


['room_segment_[0-160]', 'room_segment_[160-260]', 'room_segment_[260-360]', 'room_segment_[360-500]', 'room_segment_[500-900]']


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object for later use in the recommend method. Overwrite the users_df variable. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real interactions and equal to 0 for those newly added interactions. Generate several negative interactions per every positive interactions (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and use initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, Neural Networks or any model of your choice.
  - A different approach where you treat each item as a class, you train directly on categorical features of items and users (you would have to design appropriate categorical features for users) and you fit classifiers (e.g. Decision Tree classifier, Naive Bayes classifier etc.) instead of regressors.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender

# n_items=768
# n_users=14503

class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def generate_negative_interactions_list(self, n):
        n_items = np.max(interactions_df['item_id']) + 1
        n_users = np.max(interactions_df['user_id']) + 1

        r = np.zeros(shape=(n_users, n_items))
        for idx, interaction in interactions_df.iterrows():
            r[int(interaction['user_id'])][int(interaction['item_id'])] = 1

        negative_interactions = []

        for i1, i2 in enumerate(r):
            for j1, j2 in enumerate(i2):
                if j2 == 0:
                    negative_interactions.append((i1, j1, 0))
                    
        return random.sample(negative_interactions, n)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features] # <- The column label 'item_id' is not unique.
#         items_df = items_df.loc[:, item_features] 
        

    
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        # negative_interactions = []
        
        # Write your code here
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        #  ---start---

        
        negative_interactions = self.generate_negative_interactions_list(self.n_neg_per_pos * len(interactions_df))

        
        #  ---end---
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        # display(HTML(interactions_df.head(15).to_html()))
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # Write your code here
        # Prepare users_df and items_df
        # For users_df you just need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations)
        # For items you have to apply the prepare_items_df method to items_df.
        #  ---start---
        
                
        users_df = pd.merge(self.users_df, users_df, on=['user_id'])

        
        items_df, item_features = prepare_items_df(items_df)

        #  ---end---
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        iteration = 0;
        
        for ix, user in users_df.iterrows():
            
            # ----- start-counter -----
            iteration = iteration + 1
            print('iteration=' + str(iteration))
            print('ix=' + str(ix))
            # ----- end-counter -----
                  
            # Write your code here
            # Create a Carthesian product of users from users_df and items from items_df
            #  ---start---
            
            # carthesian_product = users_df.merge(items_df, how='cross')
            # e.g.
            # product = (
            # df1.assign(key=1)
            # merge(df2.assign(key=1), on="key")
            # .drop("key", axis=1)
            # )
            print('step-1')
            carthesian_product = (users_df.assign(key=1)
                                  .merge(items_df
                                         .assign(key=1), on='key')
                                  .drop(columns=['user_id', 'item_id', 'key'], axis=1))

#             carthesian_product = itertools.product(users_df, items_df)
            
#             carthesian_product = []
#             for i1, i2 in enumerate(users_features):
#                 for j1, j2 in enumerate(items_features):
#                     carthesian_product.append([i1, j1])
#             carthesian_product = pd.merge(users_df, items_df, on='item_id', how='cross').drop(columns=['user_id', 'item_id'])
#             print(carthesian_product)
            
            

                
                
            #  ---end---
            
            # Write your code here
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            #  ---start---
            print('step-2')
#             scores = self.model.predict(carthesian_product)
            scores = self.model.predict(carthesian_product)
            #  ---end---
            
            # Write your code here
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            #  ---start---
            # chosen_ids = None
            print('step-3')
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            #  ---end---
            print('step-4')
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])
            
                  
        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)   
      
    
# contentBasedUserItemRecommender = ContentBasedUserItemRecommender()

# print(contentBasedUserItemRecommender.generate_negative_interactions_list())
# print(len(negative_interactions))
# print(carthesian_product)

# Quick test of the recommender

In [6]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [7]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [8]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=5
ix=4
step-1
step-2
step-3
step-4
iteration=6
ix=5
step-1
step-2
step-3
step-4
iteration=7
ix=6
step-1
step-2
step-3
step-4
iteration=8
ix=7
step-1
step-2
step-3
step-4
iteration=9
ix=8
step-1
step-2
step-3
step-4
iteration=10
ix=9
step-1
step-2
step-3
step-4
iteration=11
ix=10
step-1
step-2
step-3
step-4
iteration=12
ix=11
step-1
step-2
step-3
step-4
iteration=13
ix=12
step-1
step-2
step-3
step-4
iteration=14
ix=13
step-1
step-2
step-3
step-4
iteration=15
ix=14
step-1
step-2
step-3
step-4
iteration=16
ix=15
step-1
step-2
step-3
step-4
iteration=17
ix=16
step-1
step-2
step-3
step-4
iteration=18
ix=17
step-1
step-2
step-3
step-4
iteration=19
ix=18
step-1
step-2
step-3
step-4
iteration=20
ix=19
step-1
step-2
step-3
step-4
iteration=21
ix=20
step-1
step-2
step-3
step-4
iteration=22
ix=21
step-1
step-2

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,225427,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
1,1,238414,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
2,1,388806,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
3,1,238412,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
4,1,238411,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
5,1,238410,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
6,1,388807,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
7,1,238408,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
8,1,238407,0.93417,NaN,NaN,NaN,NaN,NaN,NaN
9,1,388808,0.93417,NaN,NaN,NaN,NaN,NaN,NaN


# Tuning method

In [9]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [10]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(HTML(results.to_html()))
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [11]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|                                                                                                     | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: can't multiply sequence by non-int of type 'float'



  0%|                                                                                                     | 0/10 [00:05<?, ?trial/s, best loss=?]

Traceback (most recent call last):
  File "<ipython-input-10-164530239fdf>", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 507, in fmin
    return trials.fmin(
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\base.py", line 682, in fmin
    return fmin(
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 553, in fmin
    rval.exhaust()
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 356, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 292, in run
    self.serial_evaluate()
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 170, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\base.py", line 907, in evaluate


Best parameters:
None


In [12]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|                                                                                                    | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: can't multiply sequence by non-int of type 'float'



  0%|                                                                                                    | 0/100 [00:05<?, ?trial/s, best loss=?]

Traceback (most recent call last):
  File "<ipython-input-10-164530239fdf>", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 507, in fmin
    return trials.fmin(
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\base.py", line 682, in fmin
    return fmin(
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 553, in fmin
    rval.exhaust()
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 356, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 292, in run
    self.serial_evaluate()
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\fmin.py", line 170, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "D:\Anaconda3\Anaconda3\lib\site-packages\hyperopt\base.py", line 907, in evaluate


Best parameters:
None


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [13]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 7})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(linear_cbui_tts_results.to_html()))

iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=5
ix=4
step-1
step-2
step-3
step-4
iteration=6
ix=5
step-1
step-2
step-3
step-4
iteration=7
ix=6
step-1
step-2
step-3
step-4
iteration=8
ix=7
step-1
step-2
step-3
step-4
iteration=9
ix=8
step-1
step-2
step-3
step-4
iteration=10
ix=9
step-1
step-2
step-3
step-4
iteration=11
ix=10
step-1
step-2
step-3
step-4
iteration=12
ix=11
step-1
step-2
step-3
step-4
iteration=13
ix=12
step-1
step-2
step-3
step-4
iteration=14
ix=13
step-1
step-2
step-3
step-4
iteration=15
ix=14
step-1
step-2
step-3
step-4
iteration=16
ix=15
step-1
step-2
step-3
step-4
iteration=17
ix=16
step-1
step-2
step-3
step-4
iteration=18
ix=17
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3


iteration=58
ix=57
step-1
step-2
step-3
step-4
iteration=59
ix=58
step-1
step-2
step-3
step-4
iteration=60
ix=59
step-1
step-2
step-3
step-4
iteration=61
ix=60
step-1
step-2
step-3
step-4
iteration=62
ix=61
step-1
step-2
step-3
step-4
iteration=63
ix=62
step-1
step-2
step-3
step-4
iteration=64
ix=63
step-1
step-2
step-3
step-4
iteration=65
ix=64
step-1
step-2
step-3
step-4
iteration=66
ix=65
step-1
step-2
step-3
step-4
iteration=67
ix=66
step-1
step-2
step-3
step-4
iteration=68
ix=67
step-1
step-2
step-3
step-4
iteration=69
ix=68
step-1
step-2
step-3
step-4
iteration=70
ix=69
step-1
step-2
step-3
step-4
iteration=71
ix=70
step-1
step-2
step-3
step-4
iteration=72
ix=71
step-1
step-2
step-3
step-4
iteration=73
ix=72
step-1
step-2
step-3
step-4
iteration=74
ix=73
step-1
step-2
step-3
step-4
iteration=75
ix=74
step-1
step-2
step-3
step-4
iteration=76
ix=75
step-1
step-2
step-3
step-4
iteration=77
ix=76
step-1
step-2
step-3
step-4
iteration=78
ix=77
step-1
step-2
step-3
step-4
iteration=79


step-3
step-4
iteration=230
ix=229
step-1
step-2
step-3
step-4
iteration=231
ix=230
step-1
step-2
step-3
step-4
iteration=232
ix=231
step-1
step-2
step-3
step-4
iteration=233
ix=232
step-1
step-2
step-3
step-4
iteration=234
ix=233
step-1
step-2
step-3
step-4
iteration=235
ix=234
step-1
step-2
step-3
step-4
iteration=236
ix=235
step-1
step-2
step-3
step-4
iteration=237
ix=236
step-1
step-2
step-3
step-4
iteration=238
ix=237
step-1
step-2
step-3
step-4
iteration=239
ix=238
step-1
step-2
step-3
step-4
iteration=240
ix=239
step-1
step-2
step-3
step-4
iteration=241
ix=240
step-1
step-2
step-3
step-4
iteration=242
ix=241
step-1
step-2
step-3
step-4
iteration=243
ix=242
step-1
step-2
step-3
step-4
iteration=244
ix=243
step-1
step-2
step-3
step-4
iteration=245
ix=244
step-1
step-2
step-3
step-4
iteration=246
ix=245
step-1
step-2
step-3
step-4
iteration=247
ix=246
step-1
step-2
step-3
step-4
iteration=248
ix=247
step-1
step-2
step-3
step-4
iteration=249
ix=248
step-1
step-2
step-3
step-4
iterat

iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=5
ix=4
step-1
step-2
step-3
step-4
iteration=6
ix=5
step-1
step-2
step-3
step-4
iteration=7
ix=6
step-1
step-2
step-3
step-4
iteration=8
ix=7
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=

iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=

iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=2
ix=1
step-1
step-2
step-3
step-4
iteration=3
ix=2
step-1
step-2
step-3
step-4
iteration=4
ix=3
step-1
step-2
step-3
step-4
iteration=5
ix=4
step-1
step-2
step-3
step-4
iteration=6
ix=5
step-1
step-2
step-3
step-4
iteration=1
ix=0
step-1
step-2
step-3
step-4
iteration=

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.0,0.000658,0.000658,0.001645,0.0,0.000329,0.000329,0.000658


In [14]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(amazon_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.042448,0.104969,0.140836,0.200066,0.042448,0.077155,0.092126,0.111135


In [15]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(HTML(tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.000000,0.000658,0.000658,0.001645,0.000000,0.000329,0.000329,0.000658
1,AmazonRecommender,0.042448,0.104969,0.140836,0.200066,0.042448,0.077155,0.092126,0.111135
